In [34]:
## import of the pandas library
## questions for myself: could this be done recursively? Get the books on one level and then go to other levels? Transitive...
import pandas as pd
import pickle
import re

In [47]:
# load data
with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_data.pkl', 'rb') as f:
    books_data = pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/ratings_data.pkl', 'rb') as f:
    ratings_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/users_data.pkl', 'rb') as f:
    users_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_ratings.pkl', 'rb') as f:
    books_ratings= pickle.load(f)


In [61]:
# define a function of recommendation

def recommendation (BookIsLiked, BookTitle, BookAuthor):

    if isinstance(BookIsLiked, (float, int)):
         # give the inputs the right structure
        BookTitle = BookTitle.upper()
        BookAuthor = re.sub('. ', '.', BookAuthor.upper())

        ## find the Lord of the Rings books
        filtered_books = books_data.loc[(books_data["Book-Title"].str.contains(BookTitle)) & (books_data["Book-Author"].str.contains(BookAuthor))]
        # filtered_books

        ## getting the users that like Lord of the Rings
        users_filtered = ratings_data.loc[(ratings_data['ISBN'].isin(filtered_books['ISBN'])) & (ratings_data['Book-Rating'] > BookIsLiked)]
        lord_readers = pd.unique(users_filtered['User-ID'])
        # lord_readers

        ## getting the ISBN and Book-Ratings of the books that Lord of the Rings lovers also read
        books = ratings_data[ratings_data['User-ID'].isin(lord_readers)].reindex(columns = ['ISBN', 'Book-Rating'])
        ##books

        ## keeping only the books where there is more than 5 ratings
        count = books.groupby('ISBN')['Book-Rating'].count().reset_index(name = 'Count')
        count = count.loc[count["Count"] > 5]

        ## keeping only the books where the mean of the ratings is bigger than BookIsLiked, but the book should not be Lord of the Rings book
        ratings = pd.merge(left=ratings_data,right=count, left_on="ISBN", right_on="ISBN")
        ratings = ratings.groupby('ISBN')['Book-Rating'].mean().reset_index(name = 'MeanRating').sort_values(by='MeanRating', ascending=False)
        ratings = ratings.loc[ratings["MeanRating"] > BookIsLiked]
        ratings = ratings[ratings.ISBN.isin(filtered_books["ISBN"])==False]

        ##ratings
        return books_ratings[books_ratings['ISBN'].isin(ratings['ISBN'])].sort_values(by = ['MeanRating'], ascending = False).head(10)
    
    else:
        return print('Please, insert in BookIsLike a number.')

In [56]:
## defining variables
BookIsLiked = 4
BookTitle = 'HARRY POTTER and the order'
BookAuthor = 'j. k. rowling'

In [62]:
recommendation (BookIsLiked, BookTitle, BookAuthor)

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language,MeanRating,NumberOfRatings
16190,0060248025,SHE.SILVERSTEIN,FALLING UP,1996,HARPERCOLLINS,EN,6.921053,38
5432,0439064864,J.K.ROWLING,HARRY POTTER AND THE CHAMBER OF SECRETS BOOK 2,1999,SCHOLASTIC,EN,6.611765,170
5431,0439139597,J.K.ROWLING,HARRY POTTER AND THE GOBLET OF FIRE BOOK 4,2000,SCHOLASTIC,EN,6.541237,194
3839,0439136350,J.K.ROWLING,HARRY POTTER AND THE PRISONER OF AZKABAN BOOK 3,1999,SCHOLASTIC,EN,6.467005,197
2809,0590353403,J.K.ROWLING,HARRY POTTER AND THE SORCERER'S STONE BOOK 1,1998,SCHOLASTIC,EN,6.363095,168
12762,0618002235,J.R.R.TOLKIEN,"THE TWO TOWERS THE LORD OF THE RINGS, PART 2",1999,HOUGHTON MIFFLIN COMPANY,EN,6.230769,39
12763,0618002227,J.R.R.TOLKIEN,THE FELLOWSHIP OF THE RING THE LORD OF THE RIN...,1999,HOUGHTON MIFFLIN COMPANY,EN,6.206349,63
19640,0877017883,NIC.BANTOCK,GRIFFIN &AMP; SABINE AN EXTRAORDINARY CORRESPO...,1991,CHRONICLE BOOKS,EN,6.041667,72
3623,0811802981,NIC.BANTOCK,THE GOLDEN MEAN IN WHICH THE EXTRAORDINARY COR...,1993,CHRONICLE BOOKS,EN,5.944444,36
3776,039480001X,DR.SEUSS,THE CAT IN THE HAT,1957,RANDOM HOUSE BOOKS FOR YOUNG READERS,EN,5.730769,52
